# Data Functionalization
This notebook shows how to use the Functionalize_Dataset function to easily functionalize a given dataset. See the first cell for details on the function execution.

In [ ]:
from kamodo_ccmc.flythrough.functionalize import Functionalize_Dataset
import numpy as np
help(Functionalize_Dataset)

In [ ]:
# Initialize and functionalize data of any dimension as shown.
# The example below shows the creation of the inputs, including
# example datasets with 7 dimensions. Any number of dimensions can
# be functionalized.
rng1 = np.random.RandomState(1)  # Seed the random generators differently
rng2 = np.random.RandomState(2)  # or the arrays created below will be identical.
coord_dict = {'time': {'units': 'hr', 'data': np.linspace(0., 24., 25)},
              'lon': {'units': 'deg', 'data': np.linspace(-180., 180., 12)},
              'lat': {'units': 'deg', 'data': np.linspace(-90., 90., 5)},
              'radius': {'units': 'R_E', 'data': np.linspace(0., 50., 10)},
              'nonsense': {'units': 'm/m', 'data': np.linspace(1., 15., 15)},
              'nope': {'units': 'm', 'data': np.linspace(1., 150., 25)},
              'nada': {'units': 'hPa', 'data': np.linspace(0.00005, 15000., 20)}}
var_dict = {'Test_7D': {'units': 'S', 'data': rng1.rand(25, 12, 5, 10, 15, 25, 20)},
            'Good_7D': {'units': 'mK', 'data': rng2.rand(25, 12, 5, 10, 15, 25, 20)}}
kamodo_object = Functionalize_Dataset(coord_dict, var_dict)
kamodo_object

In [ ]:
# Plot a 1D slice of all the variables by choosing a slice value in all but one dimension.
kamodo_object.plot('Test_7D', 'Good_7D', plot_partial={
    'Test_7D': {'time': 12., 'lon': 0.5, 'lat': -20., 'radius': 15., 'nonsense': 11.5, 'nope': 5.},
    'Good_7D': {'time': 12., 'lon': 0.5, 'lat': -20., 'radius': 15., 'nonsense': 11.5, 'nope': 5.}})

In [ ]:
# Plot a 2D slice of one variables by choosing a slice value in all but two dimensions.
kamodo_object.plot('Test_7D', plot_partial={
    'Test_7D': {'time': 12., 'lon': 0.5, 'lat': -20., 'radius': 15., 'nonsense': 11.5}})

In [ ]:
# You can add datasets of other dimensions to the same kamodo_object.
coord_dict = {'time': {'units': 'hr', 'data': np.linspace(0., 24., 25)}}
var_dict = {'Test_1D': {'units': 'S', 'data': rng1.rand(25)},
            'Good_1D': {'units': 'mK', 'data': rng2.rand(25)}}
kamodo_object = Functionalize_Dataset(coord_dict, var_dict, kamodo_object)
kamodo_object

In [ ]:
# Use the get_defaults function to retrieve the coordinate arrays
# This is useful when it is not obvious what value to choose for slicing.
from kamodo import get_defaults
get_defaults(kamodo_object.Test_1D)

In [ ]:
# You can plot all of the functions on the same plot as long as the independent
# variable is the same (time in this example).
kamodo_object.plot('Test_1D', 'Good_1D', 'Test_7D', 'Good_7D', plot_partial={
    'Test_7D': {'lon': 0.5, 'lat': -20., 'radius': 15., 'nonsense': 11.5, 'nope': 5., 'nada': 12.},
    'Good_7D': {'lon': 0.5, 'lat': -20., 'radius': 15., 'nonsense': 11.5, 'nope': 5., 'nada': 12.}})

In [ ]:
# Access the metadata
kamodo_object['Test_1D'].meta

In [ ]:
# Add to the metadata
kamodo_object['Test_1D'].meta['description'] = 'Testing the functionalize.py script'
kamodo_object['Test_1D'].meta['citation'] = 'Ringuette et al. 2022'
kamodo_object['Test_1D'].meta

In [ ]:
kamodo_object.detail()